#Validacion
#Test de Sync
#**Fase 1 | Avance de Proyecto**

###**Maestría en Inteligencia Artificial Aplicada**
###**Curso: Operaciones de Aprendizaje Automático**
####**Tecnológico de Monterrey**
#####**Prof L José Ángel Martínez Navarro**

#Prueba de Sync 2

#Prueba de Sync 3 Luz

#Prueba de Sync 3 From Google

#Prueba de Sync 4 From Google

#Prueba de Sync 5 From Javy Visual Studio Code

#Prueba de Sync 6 From Javy Visual Studio Code

###**Equipo 24:**

###
A01796937 - Sandra Luz Cervantes Espinoza

A01226881 - Héctor Jesús López Meza

A01796697 - Mauricio Torres Baena

A01795838 - Javier Augusto Rebull Saucedo

A01360416 – David Cruz Beltrán

**Información:**

*   Dataset Turkish Music Emotion archivo "Acoustic Features.csv".


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from scipy import stats
from sklearn.metrics  import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [ ]:
# Cargamos desde PC
from google.colab import files
uploaded = files.upload()

df = pd.read_csv("Acoustic Features.csv")
df.head()

#**Proceso**

**Análisis exploratorio (EDA):**

*   Análisis descriptivo
* Análisis de variables numéricas
* Análisis de variables de texto
* Análisis de correlación (bivariante y multivariante)

**Preprocesamiento**
* Valores faltantes
* Valores atípicos

**Ingeniería de características**
**Entrenando y evaluando el modelo**


#**1. Análisis exploratorio (EDA)**

Estadísticas descriptivas

In [ ]:
estadisticas_descriptivas = df.describe()
estadisticas_descriptivas

Valores únicos por variable para identificar posibles variables categóricas

In [ ]:
data_df_unique = df.nunique()
data_df_unique

Búsqueda de valores faltantes

In [ ]:
data_df_missing = df.isnull().sum()
data_df_missing

Diagrama de barras para determinar la frecuencia de las emociones

In [ ]:
sns.countplot(x='Class', data=df)
plt.show()


Histogramas para cada característica numérica, para ver qué tan equilibrados están los datos

In [ ]:
hist = df.hist(bins=30, sharey=True, figsize=(16, 12))
plt.tight_layout(pad=3.0)
plt.show()

Tablas de frecuencia para cada característica categórica

In [ ]:
for column in df.select_dtypes(include=[ 'object', 'bool']).columns:
  display(column, pd.crosstab(index=df[column], columns='% observations', normalize='columns') * 100)

In [ ]:
label_encoder = LabelEncoder()
df["Class"] = label_encoder.fit_transform(df['Class'])
df

In [ ]:
df.corr()

mapa de calor que cuantifique la correlación de las variables numéricas en el dataframe

In [ ]:
data_df_numeric = df.select_dtypes(include=[np.number])

data_df_corr = data_df_numeric.corr()

plt.figure(figsize=(18,12))
sns.heatmap(data_df_corr, annot=True, linewidths=0.5)
plt.show()

#**Ingeniería de características**

Media, mediana, desviación estándar por clase

In [ ]:
if 'Class' in df.columns:
    numeric_cols = df.select_dtypes(include=[float, int]).columns
    numeric_cols = numeric_cols.drop('Class', errors='ignore')
    for col in numeric_cols[:5]:
        plt.figure(figsize=(6, 3))
        sns.boxplot(x='Class', y=col, data=df)
        plt.title(f"Boxplot de {col} por clase")
        plt.tight_layout()
        plt.show()
else:
    print("La columna 'Class' no existe en el DataFrame.")


Histogramas

In [ ]:
for col in numeric_cols[:5]:
    plt.figure(figsize=(5,3))
    sns.kdeplot(data=df, x=col, hue="Class", fill=True)
    plt.title(f"Densidad de {col}")
    plt.tight_layout()
    plt.show()


Correlaciones

#**Detección de valores atípicos (outliers)**

Z‑score: marcar valores cuyo z > 3 o < −3:

In [ ]:
z_scores = np.abs(stats.zscore(df[numeric_cols], nan_policy='omit'))
outlier_mask = (z_scores > 3)
outlier_counts = outlier_mask.sum(axis=0)
outlier_counts_series = pd.Series(outlier_counts, index=numeric_cols)
outlier_counts_series.sort_values(ascending=False).head(10)


IQR (Interquartile Range)

In [ ]:
Q1 = df[numeric_cols].quantile(0.25)
Q3 = df[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

is_outlier = ((df[numeric_cols] < (Q1 - 1.5 * IQR)) |
              (df[numeric_cols] > (Q3 + 1.5 * IQR)))
# cuenta
is_outlier.sum().sort_values(ascending=False).head(10)

#**Limpieza y tratamiento de datos**

Asegurar que la columna Class solo contiene las etiquetas esperadas y asegurar que las columnas numéricas no tienen valores “inf“, “NaN” u otros artefactos

In [ ]:
print(df["Class"].unique())

Revisar duplicados


In [ ]:
dup_count = df.duplicated().sum()
print("Duplicados:", dup_count)
if dup_count > 0:
    df = df.drop_duplicates()


TODO: Eliminar columnas constantes (desviación cero)

In [ ]:
stds = df[numeric_cols].std()
zero_std = stds[stds == 0].index.tolist()
print("Columnas constantes:", zero_std)
df = df.drop(columns=zero_std)

Version dataset limpio

In [ ]:
df_original.to_csv("turkish_music_emotion_raw.csv", index=False)
df.to_csv("turkish_music_emotion_cleaned.csv", index=False)

In [ ]:
X = df.loc[:,df.columns != "Class"]
Y = df.loc[:,df.columns == "Class"]
X

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3,random_state=42)

In [ ]:
standardscaler = StandardScaler()
x_train = standardscaler.fit_transform(X_train)
x_test = standardscaler.transform(X_test)

**Logistic Regression**

In [ ]:
logrregression = LogisticRegression(random_state=0)
logrregression.fit(x_train,Y_train.values.ravel())
y_pred = logrregression.predict(x_test)
print(y_pred)
print(Y_test)

In [ ]:
accuracy = accuracy_score(Y_test,y_pred)
print(accuracy)

In [ ]:
cm = confusion_matrix(Y_test,y_pred)
cm

**KNN Classification**

In [ ]:
model = KNeighborsClassifier()
model.fit(x_train, Y_train.values.ravel())
y_pred = model.predict(x_test)
print(accuracy_score(Y_test,y_pred))

In [ ]:
cm = confusion_matrix(Y_test,y_pred)
cm

In [ ]:
model = KNeighborsClassifier(metric="manhattan")
model.fit(x_train, Y_train.values.ravel())
y_pred = model.predict(x_test)
print(accuracy_score(Y_test,y_pred))

In [ ]:
cm = confusion_matrix(Y_test,y_pred)
cm

**Support Vector Machine**

In [ ]:
svc = SVC(kernel='linear')
svc.fit(x_train,Y_train.values.ravel())
y_pred = svc.predict(x_test)
print(accuracy_score(Y_test,y_pred))

In [ ]:
cm = confusion_matrix(Y_test,y_pred)
cm

In [ ]:
svc = SVC(kernel='rbf')
svc.fit(x_train,Y_train.values.ravel())
y_pred = svc.predict(x_test)
print(accuracy_score(Y_test,y_pred))

In [ ]:
cm = confusion_matrix(Y_test,y_pred)
cm

**Gaussian Naive Bayes**

In [ ]:
gnb = GaussianNB()
gnb.fit(x_train,Y_train.values.ravel())
y_pred = gnb.predict(x_test)
print(accuracy_score(Y_test,y_pred))

In [ ]:
cm = confusion_matrix(Y_test,y_pred)
cm

**Decision Tree**

In [ ]:
dtc = DecisionTreeClassifier(criterion="entropy")
dtc.fit(x_train,Y_train.values.ravel())
y_pred = dtc.predict(x_test)
print(accuracy_score(Y_test,y_pred))

In [ ]:
cm = confusion_matrix(Y_test,y_pred)
cm

**Random Forest**

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(x_train,Y_train.values.ravel())
y_pred = rfc.predict(x_test)
print(accuracy_score(Y_test,y_pred))

In [ ]:
cm = confusion_matrix(Y_test,y_pred)
cm

**Dimensionality Reduction**

In [ ]:
pca = PCA(n_components=8)
pca.fit(X)
x_pca = pca.transform(X)
transformed = pd.DataFrame(x_pca)
X=transformed
X

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3,random_state=42)

In [ ]:
x_train = standardscaler.fit_transform(X_train)
x_test = standardscaler.transform(X_test)

In [ ]:
classifier = KNeighborsClassifier()
classifier.fit(x_train, Y_train.values.ravel())

In [ ]:
y_pred = classifier.predict(x_test)
print(accuracy_score(Y_test,y_pred))

**Hyperparameter Optimization**

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(solver='lbfgs', max_iter=1000))
])

param_grid = {'classifier__C': [0.001, 0.01, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 10, 100]}

grid_search = GridSearchCV(pipeline, param_grid, cv=10, scoring='accuracy')
grid_search.fit(X_train_scaled, Y_train)

# Get the best hyperparameters and accuracy
best_C = grid_search.best_params_['classifier__C']
best_accuracy = grid_search.best_score_
print("Best C:", best_C)
print("Best Accuracy:", best_accuracy)


**Model Evaluation**

In [ ]:
TODO: El mejor modelo es